<a href="https://colab.research.google.com/github/dgullate/Curso-IE/blob/master/IMDB_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading and exploring the data

In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import textblob

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import re
import pickle
from scipy.sparse import hstack

from sklearn import preprocessing, model_selection, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, make_scorer,roc_curve, roc_auc_score

import os

In [0]:
train_data_path='https://github.com/dgullate/Curso-IE/blob/master/Week_4/IMDB/data/labeledTrainData.tsv?raw=true'
test_data_path='https://github.com/dgullate/Curso-IE/blob/master/Week_4/IMDB/data/testData.tsv?raw=true'
unlabeled_data_path='https://github.com/dgullate/Curso-IE/blob/master/Week_4/IMDB/data/unlabeledTrainData.tsv?raw=true'
train = pd.read_csv(train_data_path, header=0, delimiter="\t", quoting=3)
raw_train_data_unlabeled = pd.read_csv(unlabeled_data_path, header=0, delimiter="\t", quoting=3)
raw_test_data = pd.read_csv(test_data_path, header=0, delimiter="\t", quoting=3)

In [0]:
print(train.head())


In [0]:
train.shape

## Data Cleaning and Text Preprocessing


### Removing HTML Markup: The BeautifulSoup Package

In [0]:
# Initialize the BeautifulSoup object on a single movie review     
example1 = BeautifulSoup(train["review"][0])  

# Print the raw review and then the output of get_text(), for 
# comparison
print(train["review"][0])
print (example1.get_text())

Calling ```get_text()``` gives you the text of the review, without tags or markup. If you browse the BeautifulSoup documentation, you'll see that it's a very powerful library - more powerful than we need for this dataset. However, it is not considered a reliable practice to remove markup using regular expressions, so even for an application as simple as this, it's usually best to use a package like BeautifulSoup.

### Dealing with Punctuation, Numbers and Stopwords: NLTK and regular expressions
When considering how to clean the text, we should think about the data problem we are trying to solve. For many problems, it makes sense to remove punctuation. On the other hand, in this case, we are tackling a sentiment analysis problem, and it is possible that "!!!" or ":-(" could carry sentiment, and should be treated as words. In this tutorial, for simplicity, we remove the punctuation altogether, but it is something you can play with on your own.

Similarly, in this tutorial we will remove numbers, but there are other ways of dealing with them that make just as much sense. For example, we could treat them as words, or replace them all with a placeholder string such as "NUM".

In [0]:
import re
# Use regular expressions to do a find-and-replace
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search
print(letters_only)

A full overview of regular expressions is beyond the scope of this tutorial, but for now it is sufficient to know that [] indicates group membership and ^ means "not". In other words, the re.sub() statement above says, "Find anything that is NOT a lowercase letter (a-z) or an upper case letter (A-Z), and replace it with a space."

In [0]:
lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()               # Split into words

Finally, we need to decide how to deal with frequently occurring words that don't carry much meaning. Such words are called "stop words"; in English they include words such as "a", "and", "is", and "the". Conveniently, there are Python packages that come with stop word lists built in. Let's import a stop word list from the Python Natural Language Toolkit (NLTK).

In [0]:
import nltk
nltk.download('stopwords') # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english") )

This will allow you to view the list of English-language stop words. To remove stop words from our movie review, do:



In [0]:
# Remove stop words from "words"
words = [w for w in words if not w in stopwords.words("english")]
print(words)

There are many other things we could do to the data - For example, Porter Stemming and Lemmatizing (both available in NLTK) would allow us to treat "messages", "message", and "messaging" as the same word, which could certainly be useful. However, for simplicity, we will stop here.

### Putting it all together
Now we have code to clean one review - but we need to clean 25,000 training reviews! To make our code reusable, let's create a function that can be called many times:

In [0]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
  
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

Two elements here are new: First, we converted the stop word list to a different data type, a set. This is for speed; since we'll be calling this function tens of thousands of times, it needs to be fast, and searching sets in Python is much faster than searching lists.

Second, we joined the words back into one paragraph. This is to make the output easier to use in our Bag of Words, below. After defining the above function, if you call the function for a single review:

In [0]:
clean_review = review_to_words( train["review"][0] )
print(clean_review)

it should give you exactly the same output as all of the individual steps we did in preceding tutorial sections. Now let's loop through and clean all of the training set at once (this might take a few minutes depending on your computer):

In [0]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range( 0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_words( train["review"][i] ) )

In [0]:
train["clean_review"]=train["review"].map(lambda x:review_to_words(x))

In [0]:
train.head()

Now we have already prepared our training data for the next step of the process: feature extraction.

## Creating Features 
Now that we have our training reviews tidied up, how do we convert them to some kind of numeric representation for machine learning? 

### Bag of Words features (with scikit-learn)
One common approach is called a Bag of Words. The Bag of Words model learns a vocabulary from all of the documents, then models each document by counting the number of times each word appears. For example, consider the following two sentences:

Sentence 1: "The cat sat on the hat"

Sentence 2: "The dog ate the cat and the hat"

From these two sentences, our vocabulary is as follows:

{ the, cat, sat, on, hat, dog, ate, and }

To get our bags of words, we count the number of times each word occurs in each sentence. In Sentence 1, "the" appears twice, and "cat", "sat", "on", and "hat" each appear once, so the feature vector for Sentence 1 is:

{ the, cat, sat, on, hat, dog, ate, and }

Sentence 1: { 2, 1, 1, 1, 1, 0, 0, 0 }

Similarly, the features for Sentence 2 are: { 3, 1, 0, 0, 1, 1, 1, 1}

In the IMDB data, we have a very large number of reviews, which will give us a large vocabulary. To limit the size of the feature vectors, we should choose some maximum vocabulary size. Below, we use the 5000 most frequent words (remembering that stop words have already been removed).

We'll be using the feature_extraction module from scikit-learn to create bag-of-words features.

In [15]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [0]:
train_data_features[2].sum()

To see what the training data array now looks like, do:



In [0]:
print(train_data_features.shape)

It has 25,000 rows and 5,000 features (one for each vocabulary word).

Note that CountVectorizer comes with its own options to automatically do preprocessing, tokenization, and stop word removal -- for each of these, instead of specifying "None", we could have used a built-in method or specified our own function to use.  See the function documentation for more details. However, we wanted to write our own function for data cleaning in this tutorial to show you how it's done step by step.

Now that the Bag of Words model is trained, let's look at the vocabulary:

In [0]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print(len(vocab))
print(vocab)

We can also print the counts of each word in 
the vocabulary:

In [0]:
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
d={}
for tag, count in zip(vocab, dist):
    d[tag]=count
sorted_d = sorted(d.items(), key=lambda x: x[1],reverse=True)
sorted_d[:20]

## Training classifiers after feature extraction
At this point, we have numeric training features from the Bag of Words and the original sentiment labels for each feature vector, so let's do some supervised learning! We can try any classifier of our choice.

Splitting the training test into train and validation sets:

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_data_features, train["sentiment"], test_size=0.25, random_state=42)

Each line in the X_train set is a vector with 5000 integer entries, the counts of each word in the vcabulary in that text.

In [0]:
X_train.shape

### Logistic Regression

We might try as a bsaline with the simplest linear model for classification: just a logistic regression.

In [0]:
from sklearn.linear_model import LogisticRegression

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

clf=LogisticRegression()
clf.fit(X_train,y_train)
print('Accuracy on the training set: ',clf.score(X_train,y_train))
print('Accuracy on the validation set: ',clf.score(X_val,y_val))

The score on the trianing set is much better than on the validation, which is clearly a symptom of **overfitting**.
In order to mitigate this, let us include some regularization.

In [0]:
from sklearn.model_selection import GridSearchCV
parameters = {'C':[0.01,0.05,0.1],'penalty':['l1','l2']}
clf=LogisticRegression( )
grid_clf = GridSearchCV(clf, parameters, cv=3)
grid_clf.fit(X_train,y_train)

Now we can evaluate the best model and find the best parameters

In [0]:
print('Accuracy on the training set: ',grid_clf.score(X_train,y_train))
print('Accuracy on the validation set: ',grid_clf.score(X_val,y_val))

In [0]:
grid_clf.best_estimator_

We get a result of 88% on the validation set, which is quite alright.



###  Random Forest

The Random Forest algorithm is included in scikit-learn (Random Forest uses many tree-based classifiers to make predictions, hence the "forest"). Below, we set the number of trees to 100 as a reasonable default value. More trees may (or may not) perform better, but will certainly take longer to run. Likewise, the more features you include for each review, the longer this will take.

In [0]:
print ("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( X_train, y_train )

Once we have fitted the model, we can evaluate its accuracy on the validation set

In [0]:
forest.score(X_val,y_val)

We get an accuracy of 84% for the sentiment classification, which is quite ok for the simplicity of the approach.

## Creating a Submission
All that remains is to run the trained Random Forest on our test set and create a submission file. If you haven't already done so, download testData.tsv from the Data page. This file contains another 25,000 reviews and ids; our task is to predict the sentiment label.

Note that when we use the Bag of Words for the test set, we only call "transform", not "fit_transform" as we did for the training set. In machine learning, you shouldn't use the test set to fit your model, otherwise you run the risk of overfitting. For this reason, we keep the test set off-limits until we are ready to make predictions.

In [0]:
# Read the test data
test = raw_test_data

# Verify that there are 25,000 rows and 2 columns
print(test.shape)

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = grid_clf.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

In [0]:
output.head()

In [0]:
ls